In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from flask import jsonify

In [13]:
data = pd.read_csv("all_data.csv")
# convert datetime to hours (default conversion is to nanoseconds)
data['hour'] = pd.to_datetime(data.datetime).values.astype(np.int64) // 10 ** 9 / 60 / 60
# we have far too much data, so let's just get one data point per day
data['filter_hour'] = pd.to_datetime(data.datetime).dt.hour
data = data[(data.filter_hour== 12)]
data.drop(columns=['datetime', 'filter_hour'], inplace=True)
# pivot data so that it's in the structure we want
piv_data = data.pivot(index='hour', columns='station', values='PM2.5')

In [4]:
piv_data

station,Aotizhongxin,Changping,Dingling,Dongsi,Guanyuan,Gucheng,Huairou,Nongzhanguan,Shunyi,Tiantan,Wanliu,Wanshouxigong
hour,,,,,,,,,,,,
378372.0,3.0,3.0,6.0,3.0,7.0,9.0,3.0,3.0,8.0,9.0,6.0,3.0
378396.0,26.0,31.0,28.0,17.0,14.0,26.0,36.0,18.0,19.0,26.0,16.0,17.0
378420.0,91.0,110.0,75.0,107.0,104.0,102.0,95.0,106.0,95.0,105.0,101.0,101.0
378444.0,13.0,8.0,5.0,13.0,9.0,18.0,17.0,12.0,3.0,16.0,11.0,15.0
378468.0,142.0,109.0,102.0,153.0,129.0,115.0,NaN,3.0,127.0,151.0,115.0,138.0
...,...,...,...,...,...,...,...,...,...,...,...,...
413316.0,21.0,16.0,15.0,14.0,12.0,21.0,13.0,9.0,12.0,13.0,13.0,24.0
413340.0,12.0,7.0,16.0,5.0,12.0,7.0,18.0,6.0,10.0,13.0,5.0,8.0
413364.0,26.0,37.0,24.0,21.0,20.0,24.0,13.0,21.0,11.0,25.0,24.0,21.0


In [14]:
piv_data = piv_data.rolling(window=3, center=True).mean()
# piv_data.dropna()

# Concatenate Data From Different Locations

In [2]:
files = ['PRSA_Data_Aotizhongxin_20130301-20170228.csv', 'PRSA_Data_Changping_20130301-20170228.csv',
        'PRSA_Data_Dingling_20130301-20170228.csv', 'PRSA_Data_Dongsi_20130301-20170228.csv',
        'PRSA_Data_Guanyuan_20130301-20170228.csv', 'PRSA_Data_Gucheng_20130301-20170228.csv',
        'PRSA_Data_Huairou_20130301-20170228.csv', 'PRSA_Data_Nongzhanguan_20130301-20170228.csv',
        'PRSA_Data_Shunyi_20130301-20170228.csv', 'PRSA_Data_Tiantan_20130301-20170228.csv', 
        'PRSA_Data_Wanliu_20130301-20170228.csv', 'PRSA_Data_Wanshouxigong_20130301-20170228.csv']
dfs = []
for file in files:
    dfs.append(pd.read_csv(file))

In [3]:
all_data = pd.concat(dfs)
all_data.reset_index(inplace=True)
all_data['datetime'] = pd.to_datetime(all_data[['year', 'month','day', 'hour']])
all_data.drop(columns=['year', 'month','day', 'hour'], inplace=True)

In [4]:
all_data.to_csv('all_data.csv', index=False)

In [2]:
all_data = pd.read_csv('all_data.csv')
all_data

,index,No,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station,datetime
0,0,1,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,2013-03-01 00:00:00
1,1,2,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,2013-03-01 01:00:00
2,2,3,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,2013-03-01 02:00:00
3,3,4,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,2013-03-01 03:00:00
4,4,5,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,2013-03-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420763,35059,35060,11.0,32.0,3.0,24.0,400.0,72.0,12.5,1013.5,-16.2,0.0,NW,2.4,Wanshouxigong,2017-02-28 19:00:00
420764,35060,35061,13.0,32.0,3.0,41.0,500.0,50.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Wanshouxigong,2017-02-28 20:00:00
420765,35061,35062,14.0,28.0,4.0,38.0,500.0,54.0,10.8,1014.2,-13.3,0.0,NW,1.1,Wanshouxigong,2017-02-28 21:00:00
420766,35062,35063,12.0,23.0,4.0,30.0,400.0,59.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Wanshouxigong,2017-02-28 22:00:00


In [5]:
piv_data = all_data.pivot(index='datetime', columns='station', values='PM2.5')
piv_data.reset_index(inplace=True)
piv_data

station,datetime,Aotizhongxin,Changping,Dingling,Dongsi,Guanyuan,Gucheng,Huairou,Nongzhanguan,Shunyi,Tiantan,Wanliu,Wanshouxigong
0,2013-03-01 00:00:00,4.0,3.0,4.0,9.0,4.0,6.0,7.0,5.0,3.0,6.0,8.0,9.0
1,2013-03-01 01:00:00,8.0,3.0,7.0,4.0,4.0,6.0,4.0,8.0,12.0,6.0,9.0,11.0
2,2013-03-01 02:00:00,7.0,3.0,5.0,7.0,3.0,5.0,4.0,3.0,14.0,6.0,3.0,8.0
3,2013-03-01 03:00:00,6.0,3.0,6.0,3.0,3.0,6.0,3.0,5.0,12.0,6.0,11.0,8.0
4,2013-03-01 04:00:00,3.0,3.0,5.0,3.0,3.0,5.0,3.0,5.0,12.0,5.0,3.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2017-02-28 19:00:00,12.0,28.0,11.0,16.0,13.0,14.0,16.0,14.0,27.0,20.0,11.0,11.0
35060,2017-02-28 20:00:00,13.0,12.0,13.0,18.0,20.0,27.0,21.0,18.0,47.0,11.0,15.0,13.0
35061,2017-02-28 21:00:00,16.0,7.0,9.0,23.0,16.0,22.0,17.0,15.0,18.0,18.0,13.0,14.0
35062,2017-02-28 22:00:00,21.0,11.0,10.0,23.0,11.0,9.0,11.0,11.0,18.0,15.0,12.0,12.0


In [7]:
piv_data.columns

Index(['datetime', 'Aotizhongxin', 'Changping', 'Dingling', 'Dongsi',
       'Guanyuan', 'Gucheng', 'Huairou', 'Nongzhanguan', 'Shunyi', 'Tiantan',
       'Wanliu', 'Wanshouxigong'],
      dtype='object', name='station')

In [9]:
jsonify(piv_data, piv_data.columns)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.